In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from DE_library import simulate_ODE, simulate_phase_portrait, plotting
import matplotlib.pyplot as plt

import sys
from MARBLE import utils, geometry, net

rng = np.random.RandomState(1)

In [ ]:
t0, t1, dt = 0, 5, 0.05
t = np.arange(t0, t1, dt)
n = 100

X0_range = [geometry.sample_2d(n, [[-1,-1],[1,1]], 'random') for i in range(4)]

In [ ]:
fig, ax = plt.subplots(1,4, figsize=(20,5))

pos, _ = simulate_phase_portrait('bogdanov_takens', t, X0_range[0], par = {'beta1': -0.15, 'beta2': -0.6})

for p in pos:
    ax[0] = plotting.trajectories(p, ax=ax[0], style='->', lw=1, arrowhead=.03, arrow_spacing=7, axis=False, alpha=None)
ax[0].axis('square')
ax[0].set_xlim([-1,1])
ax[0].set_ylim([-1,1])

pos, _ = simulate_phase_portrait('bogdanov_takens', t, X0_range[1], par = {'beta1': -0.05, 'beta2': -0.6})

for p in pos:
    ax[1] = plotting.trajectories(p, ax=ax[1], style='->', lw=1, arrowhead=.03, arrow_spacing=7, axis=False, alpha=None)
ax[1].axis('square')
ax[1].set_xlim([-1,1])
ax[1].set_ylim([-1,1])

pos, _ = simulate_phase_portrait('bogdanov_takens', t, X0_range[2], par = {'beta1': 0.05, 'beta2': -0.6})

for p in pos:
    ax[2] = plotting.trajectories(p, ax=ax[2], style='->', lw=1, arrowhead=.03, arrow_spacing=7, axis=False, alpha=None)
ax[2].axis('square')
ax[2].set_xlim([-1,1])
ax[2].set_ylim([-1,1])

pos, _ = simulate_phase_portrait('bogdanov_takens', t, X0_range[3], par = {'beta1': 0.15, 'beta2': -0.6})

for p in pos:
    ax[3] = plotting.trajectories(p, ax=ax[3], style='->', lw=1, arrowhead=.03, arrow_spacing=7, axis=False, alpha=None)
ax[3].axis('square')
ax[3].set_xlim([-1,1])
ax[3].set_ylim([-1,1])

# Geneate phase portraits from random initial conditions while varying $\beta_1$

In [ ]:
beta1 = np.linspace(-.15,.15,20)

pos, vel = [], []
for b1 in beta1:
    X0_range = geometry.sample_2d(n, [[-1,-1],[1,1]], 'random')
    p, v = simulate_phase_portrait('bogdanov_takens', t, X0_range, par = {'beta1': b1, 'beta2': -0.6})
    p = np.vstack(p)
    v = np.vstack(v)
    pos.append(p)
    vel.append(v)

# Create random projections of state-space to scalar time series

In [ ]:
from sklearn import random_projection
transformer = random_projection.SparseRandomProjection(n_components=1)

n_obs = 5

x = []
for i in range(n_obs):
    x_tmp = transformer.fit_transform(X)
    x_tmp = utils.standardise(x_tmp, norm='max')
    x.append(x_tmp)
    
x_stacked = np.hstack(x)
df = utils.to_pandas(x_stacked)

# Subsample phase portrait

In [ ]:
pos_subsample, vel_subsample = [], []
for i in range(len(beta1)):
    ind, _ = geometry.furthest_point_sampling(pos[i], stop_crit=0.001)
    pos_subsample.append(pos[i][ind])
    vel_subsample.append(vel[i][ind])

In [ ]:
ax = plotting.trajectories(pos_subsample[0], style='o', lw=1, arrowhead=.03, arrow_spacing=7, axis=False, alpha=None)
ax.axis('square')
ax.set_xlim([-1,1])
ax.set_ylim([-1,1])

In [ ]:
k=5
data = utils.construct_dataset(pos_subsample, vel_subsample, graph_type='cknn', k=k)

In [ ]:
par = {'batch_size': 256, #batch size, this should be as large as possible
       'epochs': 3, #optimisation epochs
       'order': 2, #order of derivatives
       'depth': 0, #number of hops in neighbourhood
       'n_lin_layers': 2,
       'hidden_channels': 16, #number of internal dimensions in MLP
       'out_channels': 4,
      }

model = net(data, gauge='global', **par)
model.run_training(data)
model.evaluate(data)
emb, clusters, dist = model.cluster_and_embed(n_clusters=n_clusters)
emb_MDS = geometry.embed(dist, embed_typ='MDS')